<a href="https://colab.research.google.com/github/tharunpalla8466/Big-Data-Analytics/blob/main/PPP%20Loans%20Analysis/PPPAnalysisWithPySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EDA with PySpark

# Introduction

Analysis of PPP loans data with demographics information.

Now let's go tackle the PPP dataset and find some interesting trends using PySpark!

## Download PPP data

You can download the data from here (https://data.sba.gov/dataset/ppp-foia). We will use the first three files.

I have tried copying the link address and pasting it here (so no need to mount your Google Drive) - but the link address changes every few months, so instead we can download locally, drag to Google Drive, and use a shareable link.

In [1]:
# download the SBA data
# would be great but the link changes every quarter!


!gdown https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/2b55e11d-7e75-4bbb-b526-69a06c0c4731/download/public_150k_plus_230101.csv
!gdown https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/5f700a26-02f9-4d97-94a3-e3c2c43871eb/download/public_up_to_150k_1_230101.csv
!gdown https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/b785dfac-7d99-4bc0-9ab2-e87fe855174e/download/public_up_to_150k_2_230101.csv

Downloading...
From: https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/2b55e11d-7e75-4bbb-b526-69a06c0c4731/download/public_150k_plus_230101.csv
To: /content/public_150k_plus_230101.csv
100% 452M/452M [00:04<00:00, 101MB/s]
Downloading...
From: https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/5f700a26-02f9-4d97-94a3-e3c2c43871eb/download/public_up_to_150k_1_230101.csv
To: /content/public_up_to_150k_1_230101.csv
100% 414M/414M [00:06<00:00, 60.1MB/s]
Downloading...
From: https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/b785dfac-7d99-4bc0-9ab2-e87fe855174e/download/public_up_to_150k_2_230101.csv
To: /content/public_up_to_150k_2_230101.csv
100% 412M/412M [00:04<00:00, 102MB/s]


In [2]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


If you look left, you should be able to see the three big .csv files on the lefthand side.

## Download zipcode-level socioeconomic data
It's generally not known what census tract a business resides in, but the zipcode is almost always known. 

This repo (https://github.com/Ro-Data/Ro-Census-Summaries-By-Zipcode) has zipcode-level census attributes. I have downloaded all of the files to our local runtime on the left.

In [3]:
!gdown https://raw.githubusercontent.com/Ro-Data/Ro-Census-Summaries-By-Zipcode/master/econ.txt
!gdown https://raw.githubusercontent.com/Ro-Data/Ro-Census-Summaries-By-Zipcode/master/demo.txt
!gdown https://raw.githubusercontent.com/Ro-Data/Ro-Census-Summaries-By-Zipcode/master/housing.txt
!gdown https://raw.githubusercontent.com/Ro-Data/Ro-Census-Summaries-By-Zipcode/master/rural_urban.txt
!gdown https://raw.githubusercontent.com/Ro-Data/Ro-Census-Summaries-By-Zipcode/master/social.txt

Downloading...
From: https://raw.githubusercontent.com/Ro-Data/Ro-Census-Summaries-By-Zipcode/master/econ.txt
To: /content/econ.txt
49.7MB [00:00, 111MB/s]
Downloading...
From: https://raw.githubusercontent.com/Ro-Data/Ro-Census-Summaries-By-Zipcode/master/demo.txt
To: /content/demo.txt
26.7MB [00:00, 77.7MB/s]
Downloading...
From: https://raw.githubusercontent.com/Ro-Data/Ro-Census-Summaries-By-Zipcode/master/housing.txt
To: /content/housing.txt
49.0MB [00:00, 98.0MB/s]
Downloading...
From: https://raw.githubusercontent.com/Ro-Data/Ro-Census-Summaries-By-Zipcode/master/rural_urban.txt
To: /content/rural_urban.txt
923kB [00:00, 21.0MB/s]       
Downloading...
From: https://raw.githubusercontent.com/Ro-Data/Ro-Census-Summaries-By-Zipcode/master/social.txt
To: /content/social.txt
46.0MB [00:00, 114MB/s]


# Install Spark

In [4]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
#!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz # update this!

# unzip the spark file to the current folder
#!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.3.2-bin-hadoop3.tgz # update this!

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3" # update this!


# install findspark using pip
!pip install -q findspark

# findspark
import findspark
findspark.init()

# Start session

We start the builder pattern `SparkSession.builder` and then chain a configuration parameter that defined the application name.

Providing a useful `appName` helps you identify which programs are running on your Spark cluster.

In [5]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder\
        .appName("Project1_EDA_with_PySpark")\
        .getOrCreate()

In [6]:
# for concise script, we import all functions as F
import pyspark.sql.functions as F

## (1) Create a folder on the local runtime called 'census' and copy all of the .txt files over to this folder

In [7]:
# make a directory called census
!pwd "/content"
! mkdir "census"

/content


In [8]:
# move all of the txt files we just downloaded
! mv demo.txt social.txt econ.txt housing.txt rural_urban.txt "census" 

## (2) Create a folder on the local runtime called 'PPP' and copy all of the .csv files over to this folder
Like we did in class! Use mv and not cp.

In [9]:
# make a directory called PPP
! mkdir "PPP"

In [10]:
# move all of the csv files we just downloaded
! mv public_150k_plus_230101.csv public_up_to_150k_1_230101.csv public_up_to_150k_2_230101.csv "PPP"

# 🔴 Clean the PPP data 

## (1) Append/concatenate all of the PPP csv files together
You should end up with ~2.76M rows and 53 columns if you did it right. You only need one line of code if you use the wildcard. Make sure you print the rows and column count to check your work or points off!

In [ ]:
pppDF = spark.read.option("header","true").option("inferSchema","true").csv("/content/PPP/*")
print((pppDF.count(), len(pppDF.columns)))

In [ ]:
pppDF.show(10)

## (2) Clean the zipcode column for PPP files
There are some that are missing and there are some that have a long format (12345-6789).

We only want records where the `BorrowerZip`:
*  zip code exists (is non-null)
*  if it does exist, make return only the first 5 digits (https://www.datasciencemadesimple.com/extract-first-n-and-last-n-character-in-pyspark/)

Now answer:
* How many rows did you drop? 
* What percentage of the original data was returned? Do you think this is acceptable data quality?

In [ ]:
pppDF = pppDF.withColumn("BorrowerZip",pppDF.BorrowerZip.substr(1,5)).dropna(subset=["BorrowerZip"])
pppDF.show(10)
print((pppDF.count(), len(pppDF.columns)))

We dropped 160 rows and that is 0.057%.
The data quality acceptable.

# 🔴 Clean up ALL census files
Try to use as little code as possible. You should end up with exactly 33120 rows and ~900 columns (depending on if you dropped the dirty zip code column.)

## (1) Read the files

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType
customSchema = StructType([
    StructField("ZCTA5", StringType(), True)])

In [ ]:
zipcodesDF = spark.read.options(delimiter="\t").schema(customSchema).option("header", "true").csv("/content/census/demo.txt")
demoDF = spark.read.options(delimiter="\t").option("inferSchema", "true").option("header", "true").csv("/content/census/demo.txt")
econDF = spark.read.options(delimiter="\t").option("inferSchema", "true").option("header", "true").csv("/content/census/econ.txt")
housingDF = spark.read.options(delimiter="\t").option("inferSchema", "true").option("header", "true").csv("/content/census/housing.txt")
ruralUrbanDF = spark.read.options(delimiter="\t").option("inferSchema", "true").option("header", "true").csv("/content/census/rural_urban.txt")
socialDF = spark.read.options(delimiter="\t").option("inferSchema", "true").option("header", "true").csv("/content/census/social.txt")

In [ ]:
zipcodesDF.printSchema()

## (2) Join files together

Hint: everything except the zipcode column should be converted to float... check the schema to be sure...
* https://sparkbyexamples.com/pyspark/pyspark-join-two-or-multiple-dataframes/

In [ ]:
censusDF = demoDF.join(econDF,["ZCTA5"])\
  .join(housingDF,["ZCTA5"])\
  .join(ruralUrbanDF,["ZCTA5"])\
  .join(socialDF,["ZCTA5"])


In [ ]:
censusDF.printSchema()

In [ ]:
print(censusDF.count(),len(censusDF.columns))

In [ ]:
censusDF = censusDF.withColumnRenamed("ZCTA5","IntZips")
censusDF = zipcodesDF.join(censusDF).where(censusDF["IntZips"] == zipcodesDF["ZCTA5"].cast("int")).drop(F.col("IntZips"))
censusDF.printSchema()

In [ ]:
print(censusDF.count(),len(censusDF.columns))

## (1) Join the combined census data to the PPP data based on the zipcode 

Let's call this new file `df`. Use `df` for all future analysis questions.

In [ ]:
df = pppDF.join(censusDF, censusDF.ZCTA5 == pppDF.BorrowerZip).drop('ZCTA5')

In [ ]:
df.printSchema()
print(df.count(),len(df.columns))

In [ ]:
df.show(10)

## (2) Identify the Top 10 states that received loans

Defined as the sum of funds per State across all records. You will need to recode this column because it is categorical data. Use the upper limit of each range and make sure it is NUMERIC data.

In [ ]:
# Importing sum from pyspark to overide default sum function
from pyspark.sql.functions import sum,desc
TotalFundsbyState = df.groupBy("BorrowerState") \
  .agg(sum("CurrentApprovalAmount").alias("Funds")) \
  .sort(desc("Funds")) \
  .where(F.col('BorrowerState') != 'null')
print(TotalFundsbyState.show(10, truncate = False))
print("Top 10 States that received funding are")
for x in TotalFundsbyState.select(TotalFundsbyState["BorrowerState"]).collect()[0:10]:
  print(x[0])

## (3) Identify the Top 10 banks that loaned money
Defined as the sum of loaned funds per State across all records. Are many of the banks who loaned the most money in the same State? 

Again, make sure you are using the numeric version of this categorical column!

In [ ]:
df.show(10)

In [ ]:
# Importing sum from pyspark to overide default sum function
from pyspark.sql.functions import sum
TotalFundsbyBank = df.groupBy("ServicingLenderName","ServicingLenderState","BorrowerState")\
  .agg(sum("CurrentApprovalAmount").alias("Funds")) \
  .sort(desc("Funds"))
print(TotalFundsbyBank.show(10, truncate = False))

In [ ]:
print("Top 10 Banks that loaned money are")
for x in TotalFundsbyBank.select(TotalFundsbyBank["ServicingLenderName"]).collect()[0:10]:
  print(x[0])

In [ ]:
print("Number of Banks who loaned the money in the same State are {0}".format(TotalFundsbyBank.filter(TotalFundsbyBank["ServicingLenderState"] == TotalFundsbyBank["BorrowerState"] ).count()))
print("Number of Banks who loaned the most money are in the same State are {0}".format(TotalFundsbyBank.limit(10).filter(TotalFundsbyBank["ServicingLenderState"] == TotalFundsbyBank["BorrowerState"] ).count()))

## (4) Which banks loaned the most money to businesses that were in the same State?
You will need to do a logical statement here! For example, given that a bank is headquarted in CT, how much money was loaned to CT businesses by this bank?

Generalize to all banks and businesses where this is true, then sort from high to low (return the top 10.)

In [ ]:
print("The Banks who the most money to businesses that were in the same State are")
TotalFundsbyBank.filter(TotalFundsbyBank["ServicingLenderState"] == TotalFundsbyBank["BorrowerState"] ).show(10,truncate = False)

## (5) Which zipcodes received the most funding per capita?
This requires you to calculate the sum of funds per zipcode and then divide by the population per zipcode, then sort from high to low. 

Make sure you use the clean 5 digit zip code!

## (6) What is the top industry (NAICS codes) in the top 100 zip codes that received money?
Top 100 zip codes meaning the zip codes that borrowed the most money. I want the most common NAICS code per zip code. Show all 100 rows then sum per NAICS to describe overall trends.

This one is tricky!

## (7) What are the socioeconomic characteristics of the top 100 zip codes? How do they relate to loan amounts?

Select two census variables that look interesting to you. 

Create histograms or scatterplots of those interesting socioeconomic variables with data from the PPP. Make them beautiful and describe what you see.

Don't just pick random variables - try to find an interesting story to tell with data and motivate WHY you picked this column!

## (8) Optional extra credit for +10 points. What are the Top 10 zipcodes that had the highest per capita amount of loans going to Women-Owned businesses?

This column is called `Gender` in the `PPP` data.

* What State are these zipcodes in? 
* Comment on if they have anything in common? 

# 🔴 Conclusion (5 pts)

## (1) What did you learn? 
Write five detailed bullets about what you learned. 

# On Your Own (no credit)
No points - but can you download ALL files for PPP data (several gigabytes of data) and re-run this script with ease? Post on the discussion board if you do!